In [1]:
!pip install ipywidgets

In [2]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact_manual
#jupyter nbextension enable --py widgetsnbextension

In [3]:
data_df = pd.read_csv('data.csv')

In [4]:
temp_df = data_df[~data_df["YearFrom"].isin(["16..","17.."])]  

In [5]:
temp_df = temp_df.astype({"YearFrom": float})
# temp_df["YearFrom"] = pd.to_numeric(temp_df["YearFrom"])

In [6]:
@interact_manual
def show(column=widgets.Dropdown(
    options=["Age", "VerdachteAchternaam/AccusedLastName","Suspect Professional/AccusedProfession", "Misdrijven/Crimes"],
    value='VerdachteAchternaam/AccusedLastName',
    description='Column:'), 
    years=widgets.IntRangeSlider(
    value=[1600, 1700],
    min=1500, max=1803, step=1,
    description='Select Year:',
    continuous_update=False)
    ):
    
    if years[0]==years[1]:
        counts = temp_df[column].loc[temp_df["YearFrom"]==years[0]].value_counts()
    else:
        counts = temp_df[column].loc[temp_df["YearFrom"].isin(range(years[0], years[1], 1))].value_counts()
    try:
        counts.plot(figsize=(3, len(counts)/6),kind='barh')
    except:
        print("No data")
    return    

interactive(children=(Dropdown(description='Column:', index=1, options=('Age', 'VerdachteAchternaam/AccusedLas…